In [ ]:
# --- IMPORTS --- 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# --- FUNCTION: Find anthropometry from basic measurements ---
def anthropometry_from_basic_measurements(body_weight, thigh_length, shank_length):
    """
    Returns segment parameters for inverse dynamics given simple measurements.
    
    Inputs:
    - body_weight: total body weight in kg
    - thigh_length: thigh length (hip to knee) in meters
    - shank_length: shank length (knee to ankle) in meters
    
    Outputs:
    - l1, l2: segment lengths
    - a1, a2: COM positions from proximal end
    - m1, m2: segment masses
    - I1, I2: segment moments of inertia
    """
    
    # Mass fractions
    m1_frac = 0.105  # thigh
    m2_frac = 0.0465 # shank
    
    m1 = body_weight * m1_frac
    m2 = body_weight * m2_frac
    
    # Segment lengths
    l1 = thigh_length
    l2 = shank_length
    
    # COM locations from proximal end
    a1 = 0.43 * l1
    a2 = 0.43 * l2
    
    # Moments of inertia about COM (approx cylinder model)
    k1 = 0.322  # thigh
    k2 = 0.303  # shank
    I1 = m1 * (k1 * l1)**2
    I2 = m2 * (k2 * l2)**2
    
    return l1, l2, a1, a2, m1, m2, I1, I2

In [ ]:
# --- FUNCTION: Process IMU and GRF data ---
def process_imu_grf_data(knee_imu, ankle_imu, grf_data, fs):
    """
    Converts IMU angles and GRF measurements into inputs
    for the inverse dynamics function.
    
    Inputs:
    - knee_imu: array-like
        Knee IMU time series angles (rad)
    - ankle_imu: array-like
        Ankle/shank IMU time series angles (rad)
    - grf_data: dict
        Dictionary containing 'Fx' and 'Fz' arrays
    - fs: float
        Sampling frequency (Hz)
        
    Outputs:
    - t: np.ndarray
        Time array
    - q1, q2: np.ndarray
        Hip and knee angles for dynamics
    - dq1, dq2: np.ndarray
        Angular velocities
    - ddq1, ddq2: np.ndarray
        Angular accelerations
    - Fext: np.ndarray
        External forces array
    """
    t = np.arange(len(knee_imu)) / fs
    dt = 1/fs

    q1 = knee_imu 
    q2 = ankle_imu - knee_imu 

    # velocities & accelerations
    dq1 = np.gradient(q1, dt)
    dq2 = np.gradient(q2, dt)
    ddq1 = np.gradient(dq1, dt)
    ddq2 = np.gradient(dq2, dt)

    # external forces
    Fx = grf_data.get('Fx', np.zeros_like(grf_data['Fz']))
    Fz = grf_data['Fz']
    Fext = np.vstack([Fx, Fz])

    return t, q1, q2, dq1, dq2, ddq1, ddq2, Fext

In [ ]:
# --- FUNCTION: Detect stress peaks ---
def detect_stress_peaks(t, tau, moment_arm, area, smoothing_window=7, min_prominence=0.2):
    """ 
    Computes stress from torque, moment arm, and area time series, 
    smooths it, and detects peaks above a threshold.

    Inputs:
    - t: time array
    - tau: torque time series (N*m)
    - moment_arm: moment arm time series (m)
    - area: contact/tendon area time series (m^2)
    - smoothing_window: int, moving average window size
    - min_prominence: float, fraction of max stress for peak threshold (0..1)

    Outputs:
    - stress: raw stress time series (Pa)
    - stress_smooth: smoothed stress time series (Pa)
    - peaks: indices of detected stress peaks
    """
    tau = np.asarray(tau)
    moment_arm = np.asarray(moment_arm)
    area = np.asarray(area)

    # Compute force + stress
    F = tau / (moment_arm + 1e-12)
    stress = F / (area + 1e-12)

    # Smooth
    w = np.ones(smoothing_window) / smoothing_window
    stress_smooth = np.convolve(stress, w, mode='same')

    # Peak detection
    peaks = []
    threshold = min_prominence * np.max(stress_smooth)
    for i in range(1, len(stress_smooth)-1):
        if stress_smooth[i] > stress_smooth[i-1] and stress_smooth[i] > stress_smooth[i+1]:
            if stress_smooth[i] >= threshold:
                peaks.append(i)

    return stress, stress_smooth, np.array(peaks)

In [ ]:
# --- FUNCTION: Run inverse dynamics with stress analysis >:) ---
def run_inverse_dynamics_with_stress(t, q1, q2, dq1, dq2, ddq1, ddq2, Fext, l1, l2, a1, a2, m1, m2, I1, I2):
    """ 
    Runs inverse dynamics to compute joint torques and performs stress analysis.

    Inputs:
    - t: time array
    - q1, q2: joint angles time series
    - dq1, dq2: joint angular velocities time series
    - ddq1, ddq2: joint angular accelerations time series
    - Fext: external force array (2 x N)
    - l1, l2: segment lengths
    - a1, a2: segment COM distances
    - m1, m2: segment masses
    - I1, I2: segment moments of inertia

    Outputs:
    - df: pandas DataFrame with results
    - peaks: indices of detected stress peaks
    """
    #---- Little g ---
    g = 9.81

    # ---- Time series ----
    N = len(t)

    # ---------- Dynamics helper functions ----------
    def inertia_matrix(q2):
        d1 = I1 + I2 + m1*(a1**2) + m2*(l1**2 + a2**2)
        d2 = I2 + m2*(a2**2)
        M11 = d1 + 2*m2*l1*a2*np.cos(q2)
        M12 = d2 + m2*l1*a2*np.cos(q2)
        return np.array([[M11, M12], [M12, d2]])

    def coriolis_vector(q2, dq1, dq2):
        h = -m2*l1*a2*np.sin(q2)
        return np.array([h*(2*dq1*dq2 + dq2*dq2), h*(-dq1*dq1)])

    def gravity_vector(q1, q2):
        g1 = (m1*a1 + m2*l1)*g*np.cos(q1) + m2*a2*g*np.cos(q1+q2)
        g2 = m2*a2*g*np.cos(q1+q2)
        return np.array([g1, g2])

    def foot_jacobian(q1, q2):
        J11 = -l1*np.sin(q1) - l2*np.sin(q1 + q2)
        J12 = -l2*np.sin(q1 + q2)
        J21 =  l1*np.cos(q1) + l2*np.cos(q1 + q2)
        J22 =  l2*np.cos(q1 + q2)
        return np.array([[J11, J12], [J21, J22]])

    # ---- Compute Torques ----
    tau = np.zeros((N, 2))

    for i in range(N):
        M = inertia_matrix(q2[i])
        C = coriolis_vector(q2[i], dq1[i], dq2[i])
        G = gravity_vector(q1[i], q2[i])
        J = foot_jacobian(q1[i], q2[i])
        ddq = np.array([ddq1[i], ddq2[i]])
        tau[i] = M.dot(ddq) + C + G - J.T.dot(Fext[:, i])

    tau_hip = tau[:, 0]
    tau_knee = tau[:, 1]

    # STRESS ANALYSIS (choose knee torque as example)
    moment_arm = 0.04 + 0.01*np.sin(2*np.pi*1.5*t)   # m
    area = 432e-6                                    # m^2

    stress, stress_smooth, peaks = detect_stress_peaks(
        t=t,
        tau=tau_knee,
        moment_arm=moment_arm,
        area=area,
        smoothing_window=7,
        min_prominence=0.2
    )

    # CREATE TABLE
    df = pd.DataFrame({
        't (s)': t,
        'hip_angle (rad)': q1,
        'knee_angle_rel (rad)': q2,
        'tau_hip (Nm)': tau_hip,
        'tau_knee (Nm)': tau_knee,
        'Fz (N)': Fz,
        'moment_arm (m)': moment_arm,
        'stress (Pa)': stress,
        'stress_smooth (Pa)': stress_smooth,
        'is_peak': [1 if i in peaks else 0 for i in range(N)]
    })

    # PLOT
    plt.figure(figsize=(12,6))

    # Torques
    plt.subplot(2,1,1)
    plt.plot(t, tau_hip, label='Hip Torque')
    plt.plot(t, tau_knee, label='Knee Torque')
    plt.title("Joint Torques")
    plt.ylabel("Torque (Nm)")
    plt.grid(True)
    plt.legend()

    # Stress
    plt.subplot(2,1,2)
    plt.plot(t, stress, label="Raw Stress", alpha=0.5)
    plt.plot(t, stress_smooth, label="Smoothed Stress", linewidth=2)
    plt.scatter(t[peaks], stress_smooth[peaks], color='red', label='Peaks')
    plt.title("Knee Joint Stress (proxy)")
    plt.xlabel("Time (s)")
    plt.ylabel("Stress (Pa)")
    plt.grid(True)
    plt.legend()

    plt.tight_layout()
    plt.show()

    return df, peaks

In [ ]:
# Step 1: Get real data from IMUs + pressure sensor
knee_imu = ...      # measured knee angles
ankle_imu = ...     # measured shank angles
grf_data = {'Fz': ..., 'Fx': ...}

# Step 2: Process it into dynamics inputs
t, q1, q2, dq1, dq2, ddq1, ddq2, Fext = process_imu_grf_data(
    knee_imu, ankle_imu, grf_data, fs=200
)

In [ ]:
# Step 3: Measure and input physical constants
body_weight = # kg
thigh_length = # m
shank_length = # m

# Step 4: Process it into anthropometric parameters
l1, l2, a1, a2, m1, m2, I1, I2 = anthropometry_from_basic_measurements(body_weight, thigh_length, shank_length)

In [ ]:
# Step 5: Run inverse dynamics with stress analysis
df_output, stress_peaks = run_inverse_dynamics_with_stress(t, q1, q2, dq1, dq2, ddq1, ddq2, Fext, l1, l2, a1, a2, m1, m2, I1, I2)

# Step 6: View output table
df_output.head(10)